In [ ]:
import torch
from processing.models import Stockfish
import os
import numpy as np
from openai import OpenAI
import json
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

from processing.concept_generation import analyze_puzzle_concept_importance
from processing.static_concept_detection import get_structured_engine_evaluation
from processing.models import Stockfish

load_dotenv()
api_key = os.getenv("API_KEY")

stockfish_8_path = "Stockfish-8/Windows/stockfish_8_x64.exe"
stockfish_engine = Stockfish(path=stockfish_8_path, parameters={"Threads": 1})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using PyTorch device: {device}")

Using PyTorch device: cpu
Using PyTorch device: cpu


In [ ]:
PUZZLE_CSV_PATH = 'subset_puzzles_for_commentary.csv'
OUTPUT_JSONL_PATH = 'generated_commentaries_multi_move.jsonl' 
ONNX_MODEL_PATH = 'prepared_network_fp16.onnx' 
PROBE_SAVE_DIR = 'concepts_sequential/saved_probes' 
STOCKFISH_PATH = "Stockfish-8/Windows/stockfish_8_x64.exe" 
LLM_MODEL = "gpt-4o" 

TARGET_CONCEPTS = [
    "fork", "pin", "skewer", "discoveredAttack", "sacrifice",
    "hangingPiece", "kingsideAttack", "exposedKing", "attraction",
    "deflection", "interference", "clearance", "intermezzo",
    "advancedPawn", "attackingF2F7", "capturingDefender", "doubleCheck",
    "queensideAttack", "trappedPiece", "defensiveMove", "quietMove",
    "mate"
]
TARGET_LAYER = 10 
ACTIVATION_THRESHOLD = 0.85

# --- Initialization ---
client = OpenAI()
stockfish_engine = Stockfish(path=STOCKFISH_PATH, parameters={"Threads": 1})

base_prompt = [
{
"role": "system",
"content": """
You are an expert chess commentator generating commentary for a club-level player. Explain the short tactical sequence from the perspective of the player making the `target_move`**. \n
Your goal is to provide a concise, clear, and fluid explanation focusing **on the chess actions, spatial relationships, piece interactions, tactics, and consequences on the board.** \n

You will internally use the following information to guide your understanding:\n
- FENs before the player's first move (`target_fen_s0`) and before their follow-up move (`target_fen_s2`).\n
- The move sequence: `target_move`, `opponent_reply`, `best_move_after_s2`.\n
- Stockfish evaluation hints (`stockfish_eval`), if negative eval then black is better, if positive eval then white is better, the higher the eval, the stronger the position for the player.\n
- List of potential tactical concepts in the position: `concepts_s0` (hints for the situation before `target_move`) and `concepts_s2` (hints for the situation before `best_move_after_s2`).\n
- Use the concepts as clear hints for the position and tie the commentaries you generate to the concept examples you get, if you can spatially verify that they truly exist in the positon.\n

**Output Format:** \n
- Provide a clear line of commentary explaining the sequence of moves along with the tactics it entails.
"""
},

# FEW-SHOT EXAMPLE 1 (Black - Fork / Hanging Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r4rk1/p3ppbp/Pp1q1np1/3PpbB1/2B5/2N2P2/1PPQ2PP/3RR1K1 b - - 2 18",
  "target_move": {"san": "Qc5+", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": true},
  "opponent_reply": {"san": "Kh1", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r4rk1/p3ppbp/Pp3np1/2qPpbB1/2B5/2N2P2/1PPQ2PP/3RR2K b - - 4 19",
  "best_move_after_s2": {"san": "Qxc4", "piece_moved": "Q", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["hangingPiece],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -341}, "eval_s2": {"type": "cp", "value": -320}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black plays the sharp Qc5+, executing a fork *by attacking the King on g1 and the undefended Bishop on c4. White must address the check with Kh1, leavin the c4 Bishop hanging, which Black immediately captures with the follow-up Qxc4, winning material.
"""
},

# FEW-SHOT EXAMPLE 2 (White - Sacrifice/Discovery/Mate etc.)
{
"role": "user",
"content": """
{
  "target_fen_s0": "2r2r1k/pp1q1ppp/1b6/3p4/1P1NnN2/P3P1Q1/1B3PPP/5RK1 w - - 3 20",
  "target_move": {"san": "Qxg7+", "piece_moved": "Q", "is_capture": true, "captured_piece": "P", "gives_check": true},
  "opponent_reply": {"san": "Kxg7", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "2r2r2/pp1q1pkp/1b6/3p4/1P1NnN2/P3P3/1B3PPP/5RK1 w - - 0 21",
  "best_move_after_s2": {"san": "Nf5+", "piece_moved": "N", "is_capture": false, "gives_check": true},
  "concepts_s0": ["sacrifice", "kingsideAttack" "attraction", "clearance"],
  "concepts_s2": ["discoveredAttack", "doubleCheck", "kingsideAttack", "mate"],
  "stockfish_eval": {"eval_s0": {"type": "mate", "value": 3}, "eval_s2": {"type": "mate", "value": 2}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White initiates a brilliant kingside attack with the Queen sacrifice Qxg7+. This captures the pawn, clears the g7 square, and forces Black's King via attraction to capture with Kxg7. With the King *pulled* to g7, White delivers the stunning follow-up Nf5+. This is a lethal discovered attack *unleashed by the knight moving off the b2-g7 diagonal*, resulting in a double check from both the Knight *now on f5* and the Bishop on b2, leading directly to checkmate.
"""
},

# FEW-SHOT EXAMPLE 3 (White - Mate threat/Hanging Piece)
# {
# "role": "user",
# "content": """
# {
#   "target_fen_s0": "7Q/5r2/2p3p1/3p1pk1/3P4/2q1P3/4BKP1/8 w - - 8 45",
#   "target_move": {"san": "g3", "piece_moved": "P", "is_capture": false, "captured_piece": null, "gives_check": false},
#   "opponent_reply": {"san": "Rh7", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
#   "target_fen_s2": "7Q/7r/2p3p1/3p1pk1/3P4/2q1P1P1/4BK2/8 w - - 1 46",
#   "best_move_after_s2": {"san": "Qxh7", "piece_moved": "Q", "is_capture": true, "captured_piece": "R", "gives_check": false},
#   "concepts_s0": ["mate"],
#   "concepts_s2": [hangingPiece"],
#   "stockfish_eval": {"eval_s0": {"type": "cp", "value": 477}, "eval_s2": {"type": "cp", "value": 468}}
# }"""
# },
# {
# "role": "assistant",
# "content": """
# **Commentary:**
# White plays the subtle setup move g3. While not an immediate threat itself, this pawn push prepares a mating net by creating potential escape squares for the King and enabling Qh4#. Black is forced to defend against the immediate mate threat with Rh7. However, this blocks the h-file and leaves the Rook undefended on h7. White promptly captures the hanging Rook with Qxh7, securing a decisive material advantage.
# """
# },

# FEW-SHOT EXAMPLE 4 (White - Fork/Interference/Hanging/Capturing Defender)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r5k1/1b2nppp/p7/1p6/2p1nB2/2P2N2/PPB3PP/3R2K1 w - - 0 18",
  "target_move": {"san": "Rd7", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Ng6", "piece_moved": "N", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r5k1/1b1R1ppp/p5n1/1p6/2p1nB2/2P2N2/PPB3PP/6K1 w - - 2 19",
  "best_move_after_s2": {"san": "Rxb7", "piece_moved": "R", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork", "interference"],
  "concepts_s2": ["hangingPiece", "capturingDefender": 0.82],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 337}, "eval_s2": {"type": "cp", "value": 366}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the strong positional move Rd7, placing the rook actively on the 7th rank. This creates multiple threats: it's a fork attacking both the Knight on e7 and the Bishop on b7, *and* it also interferes with the Bishop's defense of the Knight on e4. Black chooses to save the Knight with Ng6, but this leaves the Bishop on b7 hanging. White immediately captures the undefended Bishop with Rxb7, winning material and leaving the Knight on e4 stranded without its crucial defender.
"""
},
# FEW-SHOT EXAMPLE 6 (Black - Fork / Pin)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3r1br1/4kp2/p1Qp1q2/4p3/P3P3/4B3/1PP3RP/R6K b - - 0 23",
  "target_move": {"san": "Qf3", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Rg1", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "3r1br1/4kp2/p1Qp4/4p3/P3P3/4Bq2/1PP3RP/6RK b - - 2 24",
  "best_move_after_s2": {"san": "Rxg2", "piece_moved": "R", "is_capture": true, "captured_piece": "R", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["pin"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -421}, "eval_s2": {"type": "cp", "value": -424}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black intensifies the attack with Qf3. This move creates a fork, threatening checkmate on g2 (simultaneously attacking the Rook) and also attacking the Bishop on e3. Furthermore, it creates a pin on the g2-rook against the King on g1. White is forced to defend against the mate threat with Rg1. Black exploits the pin on the g-file with the strong continuation Rxg2, capturing the now-pinned Rook, and will pick up the loose Bishop on e3 next turn.
"""
},

# FEW-SHOT EXAMPLE 7 (Black - Deflection / Advanced Pawn / Defensive Move)
{
"role": "user",
"content": """
{
  "target_fen_s0": "8/3P4/2K2k2/3Q2pp/8/5P2/3q4/8 b - - 2 55",
  "target_move": {"san": "Qxd5+", "piece_moved": "Q", "is_capture": true, "captured_piece": "Q", "gives_check": true},
  "opponent_reply": {"san": "Kxd5", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "8/3P4/5k2/3K2pp/8/5P2/8/8 b - - 0 56",
  "best_move_after_s2": {"san": "Ke7", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "concepts_s0": ["deflection"],
  "concepts_s2": ["defensiveMove", "advancedPawn"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -4134}, "eval_s2": {"type": "cp", "value": -4134}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Facing White's dangerous advanced pawn on d7, Black initiates a crucial simplifying sequence with Qxd5+. This forces a Queen trade and achieves deflection by pulling the White King to d5, *away from the key squares (c6, c7, d8) needed to support the pawn's promotion*. After the forced recapture Kxd5, Black plays the essential defensive move Ke7. This allows Black's King to directly confront and neutralize White's main threat (the d7 pawn), securing a winning endgame.
"""
},

# FEW-SHOT EXAMPLE 8 (White - Intermezzo / Trapped Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3rr1k1/1p1bqppp/p1n1pb2/3p1P2/4N3/8/PP2QBPP/1B1RR2K w - - 5 21",
  "target_move": {"san": "Bc5", "piece_moved": "B", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "dxe4", "piece_moved": "P", "is_capture": true, "captured_piece": "N", "gives_check": false},
  "target_fen_s2": "3rr1k1/1p1bqppp/p1n1pb2/2B2P2/4p3/8/PP2Q1PP/1B1RR2K w - - 0 22",
  "best_move_after_s2": {"san": "Bxe7", "piece_moved": "B", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "concepts_s0": ["intermezzo", "trappedPiece"],
  "concepts_s2": ["trappedPiece": 0.95],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 178}, "eval_s2": {"type": "cp", "value": 231}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the clever Bc5! This move acts as an intermezzo, temporarily ignoring the threat to the Knight on e4. Instead, it attacks the Black Queen on e7, trapping it *as it has no safe squares to move to (d8, f6 controlled, d6 blocked)*. Black captures the Knight with dxe4, but this doesn't solve the Queen's problem. The Queen remains trapped, and White follows through with the plan by capturing it with Bxe7, securing a significant material advantage.
"""
}
]

print(f"Loading puzzles from: {PUZZLE_CSV_PATH}")
df_puzzles = pd.read_csv(PUZZLE_CSV_PATH)

print(f"Found {len(df_puzzles)} puzzles.")
generated_count = 0

# Open output file in append mode
with open(OUTPUT_JSONL_PATH, 'a') as outfile:
    for index, row in tqdm(df_puzzles.iterrows(), total=len(df_puzzles), desc="Generating Commentaries"):
        puzzle_id = row['PuzzleId'] # Assuming a 'PuzzleId' column exists
        initial_fen = row['FEN']
        moves_str = row['Moves']
        puzzle_moves_uci = moves_str.split()

        # 1. Analyze concepts using probes
        analysis_results = analyze_puzzle_concept_importance(
            puzzle_fen=initial_fen,
            puzzle_moves_uci=puzzle_moves_uci,
            onnx_model_path=ONNX_MODEL_PATH,
            probes_dir=PROBE_SAVE_DIR,
            concept_names=TARGET_CONCEPTS,
            target_layer_index=TARGET_LAYER,
            activation_threshold=ACTIVATION_THRESHOLD
        )

        # 2. Get structured engine evaluation
        engine_eval = get_structured_engine_evaluation(
            initial_fen,
            puzzle_moves_uci,
            stockfish_engine,
            depth=20 
        )

        # 3. Prepare LLM Payload
        llm_payload = {
            "target_fen_s0": engine_eval['target_fen'],
            "target_move": engine_eval["target_move_dict"],
            "opponent_reply": engine_eval["opponent_reply_dict"],
            "target_fen_s2": engine_eval['fen_state_2'],
            "best_move_after_s2": engine_eval["best_move_after_s2_dict"],
            "concepts_s0": analysis_results['probes_active_before_player_move'],
            "concepts_s2": analysis_results['probes_active_before_followup'],
            "stockfish_eval": { # Simplified eval for LLM
                "eval_s0": engine_eval["eval_s0"],
                "eval_s2": engine_eval["eval_s2"]
            }
        }

        # 4. Construct final prompt
        current_prompt = base_prompt + [{"role": "user", "content": json.dumps(llm_payload, indent=2)}]

        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=current_prompt,
            temperature=0.01 
            )
        llm_response_content = response.choices[0].message.content
        print(llm_response_content)

        # 6. Parse LLM Response (basic split)
        commentary = llm_response_content # Assume entire response is commentary if structure fails

        # 7. Store Result
        result_record = {
            "puzzle_id": puzzle_id,
            "initial_fen": initial_fen,
            "moves_uci": puzzle_moves_uci,
            "target_move_san": engine_eval['target_move_dict'],
            "fen_s0": engine_eval['target_fen'],
            "fen_s2": engine_eval['fen_state_2'],
            "s0_active_probes": analysis_results['probes_active_before_player_move'],
            "s2_active_probes": analysis_results['probes_active_before_followup'],
            "engine_eval_s0": engine_eval["eval_s0"],
            "engine_eval_s2": engine_eval["eval_s2"],
            "llm_commentary": commentary
        }
        outfile.write(json.dumps(result_record) + '\n')
        generated_count += 1

print(f"\nFinished generation. Generated {generated_count} commentaries.")
print(f"Results saved to: {OUTPUT_JSONL_PATH}")


In [ ]:
# Specifically for demonstrating
PUZZLE_CSV_PATH = 'subset_puzzles_for_commentary.csv'
ONNX_MODEL_PATH = 'prepared_network_fp16.onnx' 
PROBE_SAVE_DIR = 'concepts_sequential/saved_probes' 
STOCKFISH_PATH = "Stockfish-8/Windows/stockfish_8_x64.exe" 
LLM_MODEL = "gpt-4o" 
TARGET_CONCEPTS = [
    "fork", "pin", "skewer", "discoveredAttack", "sacrifice",
    "hangingPiece", "kingsideAttack", "exposedKing", "attraction",
    "deflection", "interference", "clearance", "intermezzo",
    "advancedPawn", "attackingF2F7", "capturingDefender", "doubleCheck",
    "queensideAttack", "trappedPiece", "defensiveMove", "quietMove",
    "mate"
]
TARGET_LAYER = 10 
ACTIVATION_THRESHOLD = 0.85

base_prompt = [
{
"role": "system",
"content": """
You are an expert chess commentator generating commentary for a club-level player. Explain the short tactical sequence from the perspective of the player making the `target_move`**. \n
Your goal is to provide a concise, clear, and fluid explanation focusing **on the chess actions, spatial relationships, piece interactions, tactics, and consequences on the board.** \n

You will internally use the following information to guide your understanding:\n
- FENs before the player's first move (`target_fen_s0`) and before their follow-up move (`target_fen_s2`).\n
- The move sequence: `target_move`, `opponent_reply`, `best_move_after_s2`.\n
- Stockfish evaluation hints (`stockfish_eval`), if negative eval then black is better, if positive eval then white is better, the higher the eval, the stronger the position for the player.\n
- List of potential tactical concepts in the position: `concepts_s0` (hints for the situation before `target_move`) and `concepts_s2` (hints for the situation before `best_move_after_s2`).\n
- Use the concepts as clear hints for the position and tie the commentaries you generate to the concept examples you get, if you can spatially verify that they truly exist in the positon.\n

**Output Format:** \n
- Provide a clear line of commentary explaining the sequence of moves along with the tactics it entails.
"""
},

# FEW-SHOT EXAMPLE 1 (Black - Fork / Hanging Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r4rk1/p3ppbp/Pp1q1np1/3PpbB1/2B5/2N2P2/1PPQ2PP/3RR1K1 b - - 2 18",
  "target_move": {"san": "Qc5+", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": true},
  "opponent_reply": {"san": "Kh1", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r4rk1/p3ppbp/Pp3np1/2qPpbB1/2B5/2N2P2/1PPQ2PP/3RR2K b - - 4 19",
  "best_move_after_s2": {"san": "Qxc4", "piece_moved": "Q", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["hangingPiece],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -341}, "eval_s2": {"type": "cp", "value": -320}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black plays the sharp Qc5+, executing a fork *by attacking the King on g1 and the undefended Bishop on c4. White must address the check with Kh1, leavin the c4 Bishop hanging, which Black immediately captures with the follow-up Qxc4, winning material.
"""
},

# FEW-SHOT EXAMPLE 2 (White - Sacrifice/Discovery/Mate etc.)
{
"role": "user",
"content": """
{
  "target_fen_s0": "2r2r1k/pp1q1ppp/1b6/3p4/1P1NnN2/P3P1Q1/1B3PPP/5RK1 w - - 3 20",
  "target_move": {"san": "Qxg7+", "piece_moved": "Q", "is_capture": true, "captured_piece": "P", "gives_check": true},
  "opponent_reply": {"san": "Kxg7", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "2r2r2/pp1q1pkp/1b6/3p4/1P1NnN2/P3P3/1B3PPP/5RK1 w - - 0 21",
  "best_move_after_s2": {"san": "Nf5+", "piece_moved": "N", "is_capture": false, "gives_check": true},
  "concepts_s0": ["sacrifice", "kingsideAttack" "attraction", "clearance"],
  "concepts_s2": ["discoveredAttack", "doubleCheck", "kingsideAttack", "mate"],
  "stockfish_eval": {"eval_s0": {"type": "mate", "value": 3}, "eval_s2": {"type": "mate", "value": 2}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White initiates a brilliant kingside attack with the Queen sacrifice Qxg7+. This captures the pawn, clears the g7 square, and forces Black's King via attraction to capture with Kxg7. With the King *pulled* to g7, White delivers the stunning follow-up Nf5+. This is a lethal discovered attack *unleashed by the knight moving off the b2-g7 diagonal*, resulting in a double check from both the Knight *now on f5* and the Bishop on b2, leading directly to checkmate.
"""
},

# FEW-SHOT EXAMPLE 3 (White - Mate threat/Hanging Piece)
# {
# "role": "user",
# "content": """
# {
#   "target_fen_s0": "7Q/5r2/2p3p1/3p1pk1/3P4/2q1P3/4BKP1/8 w - - 8 45",
#   "target_move": {"san": "g3", "piece_moved": "P", "is_capture": false, "captured_piece": null, "gives_check": false},
#   "opponent_reply": {"san": "Rh7", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
#   "target_fen_s2": "7Q/7r/2p3p1/3p1pk1/3P4/2q1P1P1/4BK2/8 w - - 1 46",
#   "best_move_after_s2": {"san": "Qxh7", "piece_moved": "Q", "is_capture": true, "captured_piece": "R", "gives_check": false},
#   "concepts_s0": ["mate"],
#   "concepts_s2": [hangingPiece"],
#   "stockfish_eval": {"eval_s0": {"type": "cp", "value": 477}, "eval_s2": {"type": "cp", "value": 468}}
# }"""
# },
# {
# "role": "assistant",
# "content": """
# **Commentary:**
# White plays the subtle setup move g3. While not an immediate threat itself, this pawn push prepares a mating net by creating potential escape squares for the King and enabling Qh4#. Black is forced to defend against the immediate mate threat with Rh7. However, this blocks the h-file and leaves the Rook undefended on h7. White promptly captures the hanging Rook with Qxh7, securing a decisive material advantage.
# """
# },

# FEW-SHOT EXAMPLE 4 (White - Fork/Interference/Hanging/Capturing Defender)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r5k1/1b2nppp/p7/1p6/2p1nB2/2P2N2/PPB3PP/3R2K1 w - - 0 18",
  "target_move": {"san": "Rd7", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Ng6", "piece_moved": "N", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r5k1/1b1R1ppp/p5n1/1p6/2p1nB2/2P2N2/PPB3PP/6K1 w - - 2 19",
  "best_move_after_s2": {"san": "Rxb7", "piece_moved": "R", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork", "interference"],
  "concepts_s2": ["hangingPiece", "capturingDefender": 0.82],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 337}, "eval_s2": {"type": "cp", "value": 366}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the strong positional move Rd7, placing the rook actively on the 7th rank. This creates multiple threats: it's a fork attacking both the Knight on e7 and the Bishop on b7, *and* it also interferes with the Bishop's defense of the Knight on e4. Black chooses to save the Knight with Ng6, but this leaves the Bishop on b7 hanging. White immediately captures the undefended Bishop with Rxb7, winning material and leaving the Knight on e4 stranded without its crucial defender.
"""
},
# FEW-SHOT EXAMPLE 6 (Black - Fork / Pin)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3r1br1/4kp2/p1Qp1q2/4p3/P3P3/4B3/1PP3RP/R6K b - - 0 23",
  "target_move": {"san": "Qf3", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Rg1", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "3r1br1/4kp2/p1Qp4/4p3/P3P3/4Bq2/1PP3RP/6RK b - - 2 24",
  "best_move_after_s2": {"san": "Rxg2", "piece_moved": "R", "is_capture": true, "captured_piece": "R", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["pin"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -421}, "eval_s2": {"type": "cp", "value": -424}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black intensifies the attack with Qf3. This move creates a fork, threatening checkmate on g2 (simultaneously attacking the Rook) and also attacking the Bishop on e3. Furthermore, it creates a pin on the g2-rook against the King on g1. White is forced to defend against the mate threat with Rg1. Black exploits the pin on the g-file with the strong continuation Rxg2, capturing the now-pinned Rook, and will pick up the loose Bishop on e3 next turn.
"""
},

# FEW-SHOT EXAMPLE 7 (Black - Deflection / Advanced Pawn / Defensive Move)
{
"role": "user",
"content": """
{
  "target_fen_s0": "8/3P4/2K2k2/3Q2pp/8/5P2/3q4/8 b - - 2 55",
  "target_move": {"san": "Qxd5+", "piece_moved": "Q", "is_capture": true, "captured_piece": "Q", "gives_check": true},
  "opponent_reply": {"san": "Kxd5", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "8/3P4/5k2/3K2pp/8/5P2/8/8 b - - 0 56",
  "best_move_after_s2": {"san": "Ke7", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "concepts_s0": ["deflection"],
  "concepts_s2": ["defensiveMove", "advancedPawn"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -4134}, "eval_s2": {"type": "cp", "value": -4134}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Facing White's dangerous advanced pawn on d7, Black initiates a crucial simplifying sequence with Qxd5+. This forces a Queen trade and achieves deflection by pulling the White King to d5, *away from the key squares (c6, c7, d8) needed to support the pawn's promotion*. After the forced recapture Kxd5, Black plays the essential defensive move Ke7. This allows Black's King to directly confront and neutralize White's main threat (the d7 pawn), securing a winning endgame.
"""
},

# FEW-SHOT EXAMPLE 8 (White - Intermezzo / Trapped Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3rr1k1/1p1bqppp/p1n1pb2/3p1P2/4N3/8/PP2QBPP/1B1RR2K w - - 5 21",
  "target_move": {"san": "Bc5", "piece_moved": "B", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "dxe4", "piece_moved": "P", "is_capture": true, "captured_piece": "N", "gives_check": false},
  "target_fen_s2": "3rr1k1/1p1bqppp/p1n1pb2/2B2P2/4p3/8/PP2Q1PP/1B1RR2K w - - 0 22",
  "best_move_after_s2": {"san": "Bxe7", "piece_moved": "B", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "concepts_s0": ["intermezzo", "trappedPiece"],
  "concepts_s2": ["trappedPiece": 0.95],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 178}, "eval_s2": {"type": "cp", "value": 231}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the clever Bc5! This move acts as an intermezzo, temporarily ignoring the threat to the Knight on e4. Instead, it attacks the Black Queen on e7, trapping it *as it has no safe squares to move to (d8, f6 controlled, d6 blocked)*. Black captures the Knight with dxe4, but this doesn't solve the Queen's problem. The Queen remains trapped, and White follows through with the plan by capturing it with Bxe7, securing a significant material advantage.
"""
}
]

df_puzzles = pd.read_csv(PUZZLE_CSV_PATH)


# Example 1 (Great example)
initial_fen = "6rk/pp6/2n5/3ppn1p/3p4/2P2P1q/PP3QNB/R4R1K w - - 2 29"
moves_str = ["f1g1", "f5g3", "f2g3", "g8g3"]

# Example 4 (Good example of probes working well enough, and the commentary being slightly off, potentially due to the eval.)
# initial_fen = "Q7/5pk1/3qpbr1/8/5P1p/2p4P/2R4K/5R2 w - - 0 1"
# moves_str = ['c2g2','d6d3','g2g6']

engine_eval = get_structured_engine_evaluation(
    initial_fen,
    moves_str,
    stockfish_engine,
    depth=20 
)

analysis_results = analyze_puzzle_concept_importance(
    puzzle_fen=initial_fen,
    puzzle_moves_uci=moves_str,
    onnx_model_path=ONNX_MODEL_PATH,
    probes_dir=PROBE_SAVE_DIR,
    concept_names=TARGET_CONCEPTS,
    target_layer_index=TARGET_LAYER,
    activation_threshold=ACTIVATION_THRESHOLD
)

llm_payload = {
    "target_fen_s0": engine_eval['target_fen'],
    "target_move": engine_eval["target_move_dict"],
    "opponent_reply": engine_eval["opponent_reply_dict"],
    "target_fen_s2": engine_eval['fen_state_2'],
    "best_move_after_s2": engine_eval["best_move_after_s2_dict"],
    "concepts_s0": analysis_results['probes_active_before_player_move'],
    "concepts_s2": analysis_results['probes_active_before_followup'],
    "stockfish_eval": { 
        "eval_s0": engine_eval["eval_s0"],
        "eval_s2": engine_eval["eval_s2"]
    }
}


current_prompt = base_prompt + [{"role": "user", "content": json.dumps(llm_payload, indent=2)}]

response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=current_prompt,
    temperature=0.01 
    )
llm_response_content = response.choices[0].message.content
print(llm_response_content)



--- Analyzing Puzzle Concept Importance ---
  Initial FEN: 6rk/pp6/2n5/3ppn1p/3p4/2P2P1q/PP3QNB/R4R1K w - - 2 29
  Moves: ['f1g1', 'f5g3', 'f2g3', 'g8g3']
  Target Layer: 10
  Probes Dir: concepts_sequential/saved_probes
  Activation Threshold: 0.85
  Loading probes...
  Successfully loaded probes for concepts: ['fork', 'pin', 'skewer', 'discoveredAttack', 'sacrifice', 'hangingPiece', 'kingsideAttack', 'exposedKing', 'attraction', 'deflection', 'interference', 'clearance', 'intermezzo', 'advancedPawn', 'attackingF2F7', 'capturingDefender', 'doubleCheck', 'queensideAttack', 'trappedPiece', 'defensiveMove', 'quietMove', 'mate']
  Using ONNX provider: CPUExecutionProvider
  Targeting ONNX output node: '/block9/conv2/mixin' for layer index 10
  Getting activations for State 0 (FEN: 6rk/pp6/2n5/3ppn1p/3p4/2P2P1q/PP3QNB/R5RK b - - 3 29)...


c:\Users\antri\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


  Getting activations for State 2 (FEN: 6rk/pp6/2n5/3pp2p/3p4/2P2PQq/PP4NB/R5RK b - - 0 30)...
  Running probes and checking threshold...
  Concepts active in S0 (threshold >= 0.85): ['pin', 'exposedKing']
  Concepts active in S2 (threshold >= 0.85): []
  Analysis complete.
**Commentary:**
Black plays Ng3+, a tactical move that exploits the pin on the h2-bishop against the White King. This check forces White to respond, and the only viable option is Qxg3, capturing the knight. However, this leaves the White Queen vulnerable. Black capitalizes on this by playing Rxg3, capturing the Queen and further exposing White's King, significantly increasing Black's material advantage and control over the position.


In [ ]:
# Specifically for demonstrating
PUZZLE_CSV_PATH = 'subset_puzzles_for_commentary.csv'
ONNX_MODEL_PATH = 'prepared_network_fp16.onnx' 
PROBE_SAVE_DIR = 'concepts_sequential/saved_probes' 
STOCKFISH_PATH = "Stockfish-8/Windows/stockfish_8_x64.exe" 
LLM_MODEL = "gpt-4o" 
TARGET_CONCEPTS = [
    "fork", "pin", "skewer", "discoveredAttack", "sacrifice",
    "hangingPiece", "kingsideAttack", "exposedKing", "attraction",
    "deflection", "interference", "clearance", "intermezzo",
    "advancedPawn", "attackingF2F7", "capturingDefender", "doubleCheck",
    "queensideAttack", "trappedPiece", "defensiveMove", "quietMove",
    "mate"
]
TARGET_LAYER = 10 
ACTIVATION_THRESHOLD = 0.85


base_prompt = [
{
"role": "system",
"content": """
You are an expert chess commentator generating commentary for a club-level player. Explain the short tactical sequence from the perspective of the player making the `target_move`**. \n
Your goal is to provide a concise, clear, and fluid explanation focusing **on the chess actions, spatial relationships, piece interactions, tactics, and consequences on the board.** \n

You will internally use the following information to guide your understanding:\n
- FENs before the player's first move (`target_fen_s0`) and before their follow-up move (`target_fen_s2`).\n
- The move sequence: `target_move`, `opponent_reply`, `best_move_after_s2`.\n
- Stockfish evaluation hints (`stockfish_eval`), if negative eval then black is better, if positive eval then white is better, the higher the eval, the stronger the position for the player.\n
- List of potential tactical concepts in the position: `concepts_s0` (hints for the situation before `target_move`) and `concepts_s2` (hints for the situation before `best_move_after_s2`).\n
- Use the concepts as clear hints for the position and tie the commentaries you generate to the concept examples you get, if you can spatially verify that they truly exist in the positon.\n

**Output Format:** \n
- Provide a clear line of commentary explaining the sequence of moves along with the tactics it entails.
"""
},

# FEW-SHOT EXAMPLE 1 (Black - Fork / Hanging Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r4rk1/p3ppbp/Pp1q1np1/3PpbB1/2B5/2N2P2/1PPQ2PP/3RR1K1 b - - 2 18",
  "target_move": {"san": "Qc5+", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": true},
  "opponent_reply": {"san": "Kh1", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r4rk1/p3ppbp/Pp3np1/2qPpbB1/2B5/2N2P2/1PPQ2PP/3RR2K b - - 4 19",
  "best_move_after_s2": {"san": "Qxc4", "piece_moved": "Q", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["hangingPiece],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -341}, "eval_s2": {"type": "cp", "value": -320}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black plays the sharp Qc5+, executing a fork *by attacking the King on g1 and the undefended Bishop on c4. White must address the check with Kh1, leavin the c4 Bishop hanging, which Black immediately captures with the follow-up Qxc4, winning material.
"""
},

# FEW-SHOT EXAMPLE 2 (White - Sacrifice/Discovery/Mate etc.)
{
"role": "user",
"content": """
{
  "target_fen_s0": "2r2r1k/pp1q1ppp/1b6/3p4/1P1NnN2/P3P1Q1/1B3PPP/5RK1 w - - 3 20",
  "target_move": {"san": "Qxg7+", "piece_moved": "Q", "is_capture": true, "captured_piece": "P", "gives_check": true},
  "opponent_reply": {"san": "Kxg7", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "2r2r2/pp1q1pkp/1b6/3p4/1P1NnN2/P3P3/1B3PPP/5RK1 w - - 0 21",
  "best_move_after_s2": {"san": "Nf5+", "piece_moved": "N", "is_capture": false, "gives_check": true},
  "concepts_s0": ["sacrifice", "kingsideAttack" "attraction", "clearance"],
  "concepts_s2": ["discoveredAttack", "doubleCheck", "kingsideAttack", "mate"],
  "stockfish_eval": {"eval_s0": {"type": "mate", "value": 3}, "eval_s2": {"type": "mate", "value": 2}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White initiates a brilliant kingside attack with the Queen sacrifice Qxg7+. This captures the pawn, clears the g7 square, and forces Black's King via attraction to capture with Kxg7. With the King *pulled* to g7, White delivers the stunning follow-up Nf5+. This is a lethal discovered attack *unleashed by the knight moving off the b2-g7 diagonal*, resulting in a double check from both the Knight *now on f5* and the Bishop on b2, leading directly to checkmate.
"""
},

# FEW-SHOT EXAMPLE 3 (White - Mate threat/Hanging Piece)
# {
# "role": "user",
# "content": """
# {
#   "target_fen_s0": "7Q/5r2/2p3p1/3p1pk1/3P4/2q1P3/4BKP1/8 w - - 8 45",
#   "target_move": {"san": "g3", "piece_moved": "P", "is_capture": false, "captured_piece": null, "gives_check": false},
#   "opponent_reply": {"san": "Rh7", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
#   "target_fen_s2": "7Q/7r/2p3p1/3p1pk1/3P4/2q1P1P1/4BK2/8 w - - 1 46",
#   "best_move_after_s2": {"san": "Qxh7", "piece_moved": "Q", "is_capture": true, "captured_piece": "R", "gives_check": false},
#   "concepts_s0": ["mate"],
#   "concepts_s2": [hangingPiece"],
#   "stockfish_eval": {"eval_s0": {"type": "cp", "value": 477}, "eval_s2": {"type": "cp", "value": 468}}
# }"""
# },
# {
# "role": "assistant",
# "content": """
# **Commentary:**
# White plays the subtle setup move g3. While not an immediate threat itself, this pawn push prepares a mating net by creating potential escape squares for the King and enabling Qh4#. Black is forced to defend against the immediate mate threat with Rh7. However, this blocks the h-file and leaves the Rook undefended on h7. White promptly captures the hanging Rook with Qxh7, securing a decisive material advantage.
# """
# },

# FEW-SHOT EXAMPLE 4 (White - Fork/Interference/Hanging/Capturing Defender)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r5k1/1b2nppp/p7/1p6/2p1nB2/2P2N2/PPB3PP/3R2K1 w - - 0 18",
  "target_move": {"san": "Rd7", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Ng6", "piece_moved": "N", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r5k1/1b1R1ppp/p5n1/1p6/2p1nB2/2P2N2/PPB3PP/6K1 w - - 2 19",
  "best_move_after_s2": {"san": "Rxb7", "piece_moved": "R", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork", "interference"],
  "concepts_s2": ["hangingPiece", "capturingDefender": 0.82],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 337}, "eval_s2": {"type": "cp", "value": 366}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the strong positional move Rd7, placing the rook actively on the 7th rank. This creates multiple threats: it's a fork attacking both the Knight on e7 and the Bishop on b7, *and* it also interferes with the Bishop's defense of the Knight on e4. Black chooses to save the Knight with Ng6, but this leaves the Bishop on b7 hanging. White immediately captures the undefended Bishop with Rxb7, winning material and leaving the Knight on e4 stranded without its crucial defender.
"""
},
# FEW-SHOT EXAMPLE 6 (Black - Fork / Pin)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3r1br1/4kp2/p1Qp1q2/4p3/P3P3/4B3/1PP3RP/R6K b - - 0 23",
  "target_move": {"san": "Qf3", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Rg1", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "3r1br1/4kp2/p1Qp4/4p3/P3P3/4Bq2/1PP3RP/6RK b - - 2 24",
  "best_move_after_s2": {"san": "Rxg2", "piece_moved": "R", "is_capture": true, "captured_piece": "R", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["pin"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -421}, "eval_s2": {"type": "cp", "value": -424}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black intensifies the attack with Qf3. This move creates a fork, threatening checkmate on g2 (simultaneously attacking the Rook) and also attacking the Bishop on e3. Furthermore, it creates a pin on the g2-rook against the King on g1. White is forced to defend against the mate threat with Rg1. Black exploits the pin on the g-file with the strong continuation Rxg2, capturing the now-pinned Rook, and will pick up the loose Bishop on e3 next turn.
"""
},

# FEW-SHOT EXAMPLE 7 (Black - Deflection / Advanced Pawn / Defensive Move)
{
"role": "user",
"content": """
{
  "target_fen_s0": "8/3P4/2K2k2/3Q2pp/8/5P2/3q4/8 b - - 2 55",
  "target_move": {"san": "Qxd5+", "piece_moved": "Q", "is_capture": true, "captured_piece": "Q", "gives_check": true},
  "opponent_reply": {"san": "Kxd5", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "8/3P4/5k2/3K2pp/8/5P2/8/8 b - - 0 56",
  "best_move_after_s2": {"san": "Ke7", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "concepts_s0": ["deflection"],
  "concepts_s2": ["defensiveMove", "advancedPawn"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -4134}, "eval_s2": {"type": "cp", "value": -4134}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Facing White's dangerous advanced pawn on d7, Black initiates a crucial simplifying sequence with Qxd5+. This forces a Queen trade and achieves deflection by pulling the White King to d5, *away from the key squares (c6, c7, d8) needed to support the pawn's promotion*. After the forced recapture Kxd5, Black plays the essential defensive move Ke7. This allows Black's King to directly confront and neutralize White's main threat (the d7 pawn), securing a winning endgame.
"""
},

# FEW-SHOT EXAMPLE 8 (White - Intermezzo / Trapped Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3rr1k1/1p1bqppp/p1n1pb2/3p1P2/4N3/8/PP2QBPP/1B1RR2K w - - 5 21",
  "target_move": {"san": "Bc5", "piece_moved": "B", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "dxe4", "piece_moved": "P", "is_capture": true, "captured_piece": "N", "gives_check": false},
  "target_fen_s2": "3rr1k1/1p1bqppp/p1n1pb2/2B2P2/4p3/8/PP2Q1PP/1B1RR2K w - - 0 22",
  "best_move_after_s2": {"san": "Bxe7", "piece_moved": "B", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "concepts_s0": ["intermezzo", "trappedPiece"],
  "concepts_s2": ["trappedPiece": 0.95],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 178}, "eval_s2": {"type": "cp", "value": 231}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the clever Bc5! This move acts as an intermezzo, temporarily ignoring the threat to the Knight on e4. Instead, it attacks the Black Queen on e7, trapping it *as it has no safe squares to move to (d8, f6 controlled, d6 blocked)*. Black captures the Knight with dxe4, but this doesn't solve the Queen's problem. The Queen remains trapped, and White follows through with the plan by capturing it with Bxe7, securing a significant material advantage.
"""
}
]

df_puzzles = pd.read_csv(PUZZLE_CSV_PATH)

# Example 2 (Good example of probes, commentary not perfect, hallucinations)
initial_fen =  "8/1R6/p1pk4/6bp/1QP5/P7/KP6/3r2q1 b - - 2 44"
moves_str = ["g1c5", "b7d7", "d6d7", "b4c5"]

# Example 4 (Good example of probes working well enough, and the commentary being slightly off, potentially due to the eval.)
# initial_fen = "Q7/5pk1/3qpbr1/8/5P1p/2p4P/2R4K/5R2 w - - 0 1"
# moves_str = ['c2g2','d6d3','g2g6']

engine_eval = get_structured_engine_evaluation(
    initial_fen,
    moves_str,
    stockfish_engine,
    depth=20 
)

analysis_results = analyze_puzzle_concept_importance(
    puzzle_fen=initial_fen,
    puzzle_moves_uci=moves_str,
    onnx_model_path=ONNX_MODEL_PATH,
    probes_dir=PROBE_SAVE_DIR,
    concept_names=TARGET_CONCEPTS,
    target_layer_index=TARGET_LAYER,
    activation_threshold=ACTIVATION_THRESHOLD
)

llm_payload = {
    "target_fen_s0": engine_eval['target_fen'],
    "target_move": engine_eval["target_move_dict"],
    "opponent_reply": engine_eval["opponent_reply_dict"],
    "target_fen_s2": engine_eval['fen_state_2'],
    "best_move_after_s2": engine_eval["best_move_after_s2_dict"],
    "concepts_s0": analysis_results['probes_active_before_player_move'],
    "concepts_s2": analysis_results['probes_active_before_followup'],
    "stockfish_eval": { 
        "eval_s0": engine_eval["eval_s0"],
        "eval_s2": engine_eval["eval_s2"]
    }
}


current_prompt = base_prompt + [{"role": "user", "content": json.dumps(llm_payload, indent=2)}]

response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=current_prompt,
    temperature=0.01 
    )
llm_response_content = response.choices[0].message.content
print(llm_response_content)



--- Analyzing Puzzle Concept Importance ---
  Initial FEN: 8/1R6/p1pk4/6bp/1QP5/P7/KP6/3r2q1 b - - 2 44
  Moves: ['g1c5', 'b7d7', 'd6d7', 'b4c5']
  Target Layer: 10
  Probes Dir: concepts_sequential/saved_probes
  Activation Threshold: 0.85
  Loading probes...
  Successfully loaded probes for concepts: ['fork', 'pin', 'skewer', 'discoveredAttack', 'sacrifice', 'hangingPiece', 'kingsideAttack', 'exposedKing', 'attraction', 'deflection', 'interference', 'clearance', 'intermezzo', 'advancedPawn', 'attackingF2F7', 'capturingDefender', 'doubleCheck', 'queensideAttack', 'trappedPiece', 'defensiveMove', 'quietMove', 'mate']
  Using ONNX provider: CPUExecutionProvider
  Targeting ONNX output node: '/block9/conv2/mixin' for layer index 10
  Getting activations for State 0 (FEN: 8/1R6/p1pk4/2q3bp/1QP5/P7/KP6/3r4 w - - 3 45)...


c:\Users\antri\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


  Getting activations for State 2 (FEN: 8/3k4/p1p5/2q3bp/1QP5/P7/KP6/3r4 w - - 0 46)...
  Running probes and checking threshold...
  Concepts active in S0 (threshold >= 0.85): ['skewer', 'deflection']
  Concepts active in S2 (threshold >= 0.85): ['skewer', 'hangingPiece']
  Analysis complete.
**Commentary:**
White plays Rd7+, a tactical move that serves as a deflection. By checking the Black King, White forces it to move to d7, capturing the Rook. This deflection removes the King from its defensive post, leaving the Black Queen on c5 vulnerable. After Black captures the Rook with Kxd7, White capitalizes on the situation with Qxc5, capturing the Black Queen. This sequence not only wins material but also leaves Black's position significantly weakened, as the skewer threat on the c-file is resolved, and the Bishop on g5 remains hanging.


In [ ]:
# Specifically for demonstrating
PUZZLE_CSV_PATH = 'subset_puzzles_for_commentary.csv'
ONNX_MODEL_PATH = 'prepared_network_fp16.onnx' 
PROBE_SAVE_DIR = 'concepts_sequential/saved_probes' 
STOCKFISH_PATH = "Stockfish-8/Windows/stockfish_8_x64.exe" 
LLM_MODEL = "gpt-4o" 
TARGET_CONCEPTS = [
    "fork", "pin", "skewer", "discoveredAttack", "sacrifice",
    "hangingPiece", "kingsideAttack", "exposedKing", "attraction",
    "deflection", "interference", "clearance", "intermezzo",
    "advancedPawn", "attackingF2F7", "capturingDefender", "doubleCheck",
    "queensideAttack", "trappedPiece", "defensiveMove", "quietMove",
    "mate"
]
TARGET_LAYER = 10 
ACTIVATION_THRESHOLD = 0.85

base_prompt = [
{
"role": "system",
"content": """
You are an expert chess commentator generating commentary for a club-level player. Explain the short tactical sequence from the perspective of the player making the `target_move`**. \n
Your goal is to provide a concise, clear, and fluid explanation focusing **on the chess actions, spatial relationships, piece interactions, tactics, and consequences on the board.** \n

You will internally use the following information to guide your understanding:\n
- FENs before the player's first move (`target_fen_s0`) and before their follow-up move (`target_fen_s2`).\n
- The move sequence: `target_move`, `opponent_reply`, `best_move_after_s2`.\n
- Stockfish evaluation hints (`stockfish_eval`), if negative eval then black is better, if positive eval then white is better, the higher the eval, the stronger the position for the player.\n
- List of potential tactical concepts in the position: `concepts_s0` (hints for the situation before `target_move`) and `concepts_s2` (hints for the situation before `best_move_after_s2`).\n
- Use the concepts as clear hints for the position and tie the commentaries you generate to the concept examples you get, if you can spatially verify that they truly exist in the positon.\n

**Output Format:** \n
- Provide a clear line of commentary explaining the sequence of moves along with the tactics it entails.
"""
},

# FEW-SHOT EXAMPLE 1 (Black - Fork / Hanging Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r4rk1/p3ppbp/Pp1q1np1/3PpbB1/2B5/2N2P2/1PPQ2PP/3RR1K1 b - - 2 18",
  "target_move": {"san": "Qc5+", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": true},
  "opponent_reply": {"san": "Kh1", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r4rk1/p3ppbp/Pp3np1/2qPpbB1/2B5/2N2P2/1PPQ2PP/3RR2K b - - 4 19",
  "best_move_after_s2": {"san": "Qxc4", "piece_moved": "Q", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["hangingPiece],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -341}, "eval_s2": {"type": "cp", "value": -320}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black plays the sharp Qc5+, executing a fork *by attacking the King on g1 and the undefended Bishop on c4. White must address the check with Kh1, leavin the c4 Bishop hanging, which Black immediately captures with the follow-up Qxc4, winning material.
"""
},

# FEW-SHOT EXAMPLE 2 (White - Sacrifice/Discovery/Mate etc.)
{
"role": "user",
"content": """
{
  "target_fen_s0": "2r2r1k/pp1q1ppp/1b6/3p4/1P1NnN2/P3P1Q1/1B3PPP/5RK1 w - - 3 20",
  "target_move": {"san": "Qxg7+", "piece_moved": "Q", "is_capture": true, "captured_piece": "P", "gives_check": true},
  "opponent_reply": {"san": "Kxg7", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "2r2r2/pp1q1pkp/1b6/3p4/1P1NnN2/P3P3/1B3PPP/5RK1 w - - 0 21",
  "best_move_after_s2": {"san": "Nf5+", "piece_moved": "N", "is_capture": false, "gives_check": true},
  "concepts_s0": ["sacrifice", "kingsideAttack" "attraction", "clearance"],
  "concepts_s2": ["discoveredAttack", "doubleCheck", "kingsideAttack", "mate"],
  "stockfish_eval": {"eval_s0": {"type": "mate", "value": 3}, "eval_s2": {"type": "mate", "value": 2}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White initiates a brilliant kingside attack with the Queen sacrifice Qxg7+. This captures the pawn, clears the g7 square, and forces Black's King via attraction to capture with Kxg7. With the King *pulled* to g7, White delivers the stunning follow-up Nf5+. This is a lethal discovered attack *unleashed by the knight moving off the b2-g7 diagonal*, resulting in a double check from both the Knight *now on f5* and the Bishop on b2, leading directly to checkmate.
"""
},

# FEW-SHOT EXAMPLE 4 (White - Fork/Interference/Hanging/Capturing Defender)
{
"role": "user",
"content": """
{
  "target_fen_s0": "r5k1/1b2nppp/p7/1p6/2p1nB2/2P2N2/PPB3PP/3R2K1 w - - 0 18",
  "target_move": {"san": "Rd7", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Ng6", "piece_moved": "N", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "r5k1/1b1R1ppp/p5n1/1p6/2p1nB2/2P2N2/PPB3PP/6K1 w - - 2 19",
  "best_move_after_s2": {"san": "Rxb7", "piece_moved": "R", "is_capture": true, "captured_piece": "B", "gives_check": false},
  "concepts_s0": ["fork", "interference"],
  "concepts_s2": ["hangingPiece", "capturingDefender": 0.82],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 337}, "eval_s2": {"type": "cp", "value": 366}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the strong positional move Rd7, placing the rook actively on the 7th rank. This creates multiple threats: it's a fork attacking both the Knight on e7 and the Bishop on b7, *and* it also interferes with the Bishop's defense of the Knight on e4. Black chooses to save the Knight with Ng6, but this leaves the Bishop on b7 hanging. White immediately captures the undefended Bishop with Rxb7, winning material and leaving the Knight on e4 stranded without its crucial defender.
"""
},
# FEW-SHOT EXAMPLE 6 (Black - Fork / Pin)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3r1br1/4kp2/p1Qp1q2/4p3/P3P3/4B3/1PP3RP/R6K b - - 0 23",
  "target_move": {"san": "Qf3", "piece_moved": "Q", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "Rg1", "piece_moved": "R", "is_capture": false, "captured_piece": null, "gives_check": false},
  "target_fen_s2": "3r1br1/4kp2/p1Qp4/4p3/P3P3/4Bq2/1PP3RP/6RK b - - 2 24",
  "best_move_after_s2": {"san": "Rxg2", "piece_moved": "R", "is_capture": true, "captured_piece": "R", "gives_check": false},
  "concepts_s0": ["fork"],
  "concepts_s2": ["pin"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -421}, "eval_s2": {"type": "cp", "value": -424}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Black intensifies the attack with Qf3. This move creates a fork, threatening checkmate on g2 (simultaneously attacking the Rook) and also attacking the Bishop on e3. Furthermore, it creates a pin on the g2-rook against the King on g1. White is forced to defend against the mate threat with Rg1. Black exploits the pin on the g-file with the strong continuation Rxg2, capturing the now-pinned Rook, and will pick up the loose Bishop on e3 next turn.
"""
},

# FEW-SHOT EXAMPLE 7 (Black - Deflection / Advanced Pawn / Defensive Move)
{
"role": "user",
"content": """
{
  "target_fen_s0": "8/3P4/2K2k2/3Q2pp/8/5P2/3q4/8 b - - 2 55",
  "target_move": {"san": "Qxd5+", "piece_moved": "Q", "is_capture": true, "captured_piece": "Q", "gives_check": true},
  "opponent_reply": {"san": "Kxd5", "piece_moved": "K", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "target_fen_s2": "8/3P4/5k2/3K2pp/8/5P2/8/8 b - - 0 56",
  "best_move_after_s2": {"san": "Ke7", "piece_moved": "K", "is_capture": false, "captured_piece": null, "gives_check": false},
  "concepts_s0": ["deflection"],
  "concepts_s2": ["defensiveMove", "advancedPawn"],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": -4134}, "eval_s2": {"type": "cp", "value": -4134}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
Facing White's dangerous advanced pawn on d7, Black initiates a crucial simplifying sequence with Qxd5+. This forces a Queen trade and achieves deflection by pulling the White King to d5, *away from the key squares (c6, c7, d8) needed to support the pawn's promotion*. After the forced recapture Kxd5, Black plays the essential defensive move Ke7. This allows Black's King to directly confront and neutralize White's main threat (the d7 pawn), securing a winning endgame.
"""
},

# FEW-SHOT EXAMPLE 8 (White - Intermezzo / Trapped Piece)
{
"role": "user",
"content": """
{
  "target_fen_s0": "3rr1k1/1p1bqppp/p1n1pb2/3p1P2/4N3/8/PP2QBPP/1B1RR2K w - - 5 21",
  "target_move": {"san": "Bc5", "piece_moved": "B", "is_capture": false, "captured_piece": null, "gives_check": false},
  "opponent_reply": {"san": "dxe4", "piece_moved": "P", "is_capture": true, "captured_piece": "N", "gives_check": false},
  "target_fen_s2": "3rr1k1/1p1bqppp/p1n1pb2/2B2P2/4p3/8/PP2Q1PP/1B1RR2K w - - 0 22",
  "best_move_after_s2": {"san": "Bxe7", "piece_moved": "B", "is_capture": true, "captured_piece": "Q", "gives_check": false},
  "concepts_s0": ["intermezzo", "trappedPiece"],
  "concepts_s2": ["trappedPiece": 0.95],
  "stockfish_eval": {"eval_s0": {"type": "cp", "value": 178}, "eval_s2": {"type": "cp", "value": 231}}
}"""
},
{
"role": "assistant",
"content": """
**Commentary:**
White plays the clever Bc5! This move acts as an intermezzo, temporarily ignoring the threat to the Knight on e4. Instead, it attacks the Black Queen on e7, trapping it *as it has no safe squares to move to (d8, f6 controlled, d6 blocked)*. Black captures the Knight with dxe4, but this doesn't solve the Queen's problem. The Queen remains trapped, and White follows through with the plan by capturing it with Bxe7, securing a significant material advantage.
"""
}
]

df_puzzles = pd.read_csv(PUZZLE_CSV_PATH)


# Example 3 (Bad example of probes, giving bad commentary, hallucinations)
initial_fen = "r2q1rk1/pbp1bp2/1p2pn1Q/5nN1/8/P1NB4/1PP3PP/R4RK1 w - - 2 17"
moves_str = ["d3f5", "d8d4", "g1h1", "e6f5"]

# Example 4 (Good example of probes working well enough, and the commentary being slightly off, potentially due to the eval.)
# initial_fen = "Q7/5pk1/3qpbr1/8/5P1p/2p4P/2R4K/5R2 w - - 0 1"
# moves_str = ['c2g2','d6d3','g2g6']

engine_eval = get_structured_engine_evaluation(
    initial_fen,
    moves_str,
    stockfish_engine,
    depth=20 
)

analysis_results = analyze_puzzle_concept_importance(
    puzzle_fen=initial_fen,
    puzzle_moves_uci=moves_str,
    onnx_model_path=ONNX_MODEL_PATH,
    probes_dir=PROBE_SAVE_DIR,
    concept_names=TARGET_CONCEPTS,
    target_layer_index=TARGET_LAYER,
    activation_threshold=ACTIVATION_THRESHOLD
)

llm_payload = {
    "target_fen_s0": engine_eval['target_fen'],
    "target_move": engine_eval["target_move_dict"],
    "opponent_reply": engine_eval["opponent_reply_dict"],
    "target_fen_s2": engine_eval['fen_state_2'],
    "best_move_after_s2": engine_eval["best_move_after_s2_dict"],
    "concepts_s0": analysis_results['probes_active_before_player_move'],
    "concepts_s2": analysis_results['probes_active_before_followup'],
    "stockfish_eval": { 
        "eval_s0": engine_eval["eval_s0"],
        "eval_s2": engine_eval["eval_s2"]
    }
}


current_prompt = base_prompt + [{"role": "user", "content": json.dumps(llm_payload, indent=2)}]

response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=current_prompt,
    temperature=0.01 
    )
llm_response_content = response.choices[0].message.content
print(llm_response_content)



--- Analyzing Puzzle Concept Importance ---
  Initial FEN: r2q1rk1/pbp1bp2/1p2pn1Q/5nN1/8/P1NB4/1PP3PP/R4RK1 w - - 2 17
  Moves: ['d3f5', 'd8d4', 'g1h1', 'e6f5']
  Target Layer: 10
  Probes Dir: concepts_sequential/saved_probes
  Activation Threshold: 0.85
  Loading probes...
  Successfully loaded probes for concepts: ['fork', 'pin', 'skewer', 'discoveredAttack', 'sacrifice', 'hangingPiece', 'kingsideAttack', 'exposedKing', 'attraction', 'deflection', 'interference', 'clearance', 'intermezzo', 'advancedPawn', 'attackingF2F7', 'capturingDefender', 'doubleCheck', 'queensideAttack', 'trappedPiece', 'defensiveMove', 'quietMove', 'mate']
  Using ONNX provider: CPUExecutionProvider
  Targeting ONNX output node: '/block9/conv2/mixin' for layer index 10
  Getting activations for State 0 (FEN: r2q1rk1/pbp1bp2/1p2pn1Q/5BN1/8/P1N5/1PP3PP/R4RK1 b - - 0 17)...


c:\Users\antri\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


  Getting activations for State 2 (FEN: r4rk1/pbp1bp2/1p2pn1Q/5BN1/3q4/P1N5/1PP3PP/R4R1K b - - 2 18)...
  Running probes and checking threshold...
  Concepts active in S0 (threshold >= 0.85): ['clearance']
  Concepts active in S2 (threshold >= 0.85): ['sacrifice', 'attraction']
  Analysis complete.
**Commentary:**
Black plays Qd4+, a clearance move that opens up the d-file for the rook on a8, while simultaneously delivering a check. This forces White's King to move to Kh1. With the King now on h1, Black capitalizes on the position by playing exf5, capturing the Bishop on g5. This move not only wins a piece but also strengthens Black's control over the center, maintaining a strong position with the Queen and rook poised for further action.
